# Using AGNBoost with Custom (non-default) Photometric Bands

This notebook demonstrates how to use AGNBoost with photometric bands that are not included in the default models of AGNBoost. Thankfully, this is very straightforward to do with AGNBoost, and is entiely relegated to the `Catalog` class. Frist, we import the necessary modules.

In [1]:
# Set agnboost folder as root
import os

# Navigate to the repository root (parent directory of notebooks/)
os.chdir('..')

# Import necessary libraries
import numpy as np
import pandas as pd
from agnboost import dataset, model

# Set random seed for reproducibility
np.random.seed(123)

print("AGNBoost Basic Usage Tutorial")
print("=" * 40)

2025-05-31 20:52:53.834 | INFO     | agnboost.config:<module>:11 - PROJ_ROOT path is: /home/kurt/Documents/agnboost


AGNBoost Basic Usage Tutorial


To create a `Catalog` object with custom bands, you need only create a python dictionary containing the information of the bands. Note that this dictionary has very specific formatting requirements! For example, here is the default dictionary used by AGNBoost, including 7 NIRCam bands and 4 MIRI bands:

```
band_dict =   {   "jwst.nircam.F115W": {"shorthand": "F115W", "wavelength": 1.154},
                  "jwst.nircam.F150W": {"shorthand": "F150W", "wavelength": 1.501},
                  "jwst.nircam.F200W": {"shorthand": "F200W", "wavelength": 1.988},
                  "jwst.nircam.F277W": {"shorthand": "F277W", "wavelength": 2.776},
                  "jwst.nircam.F356W": {"shorthand": "F356W", "wavelength": 3.565},
                  "jwst.nircam.F410M": {"shorthand": "F410M", "wavelength": 4.083},
                  "jwst.nircam.F444W": {"shorthand": "F444W", "wavelength": 4.402},
                  "jwst.miri.F770W": {"shorthand": "F770W", "wavelength": 7.7},
                  "jwst.miri.F1000W": {"shorthand": "F1000W", "wavelength": 10.0},
                  "jwst.miri.F1500W": {"shorthand": "F1500W", "wavelength": 15.0},
                  "jwst.miri.F2100W": {"shorthand": "F2100W", "wavelength": 21.0}
            }
```

Each key in the dictionary is the name of the photometric band in your dataset. So, if you are using mock CIGALE data to train and test models, the formatting should follow the CIGALE band naming conventions (i.e., "jwst.nircam.F115W", "jwst.miri.F2100W", etc.). The values corresponding to these keys are also python dictionaries, each with a "shorthand" key and a "wavelength" key. The "shorthand" key is the desired shortname of the photometric band (for example, with CIGALE, it is cleaner to be able to name a band "F115W" instead of the full CIGALE column name "jwst.nircam.F115W"). The "wavelength" is the pivot wavelength of the photomtric band (in microns). 

So, to create a AGNBoost model that uses ONLY the NIRCam bands, we can create the following dictionary:

In [3]:
NIRCam_band_dict =   {    "jwst.nircam.F115W": {"shorthand": "F115W", "wavelength": 1.154},
                          "jwst.nircam.F150W": {"shorthand": "F150W", "wavelength": 1.501},
                          "jwst.nircam.F200W": {"shorthand": "F200W", "wavelength": 1.988},
                          "jwst.nircam.F277W": {"shorthand": "F277W", "wavelength": 2.776},
                          "jwst.nircam.F356W": {"shorthand": "F356W", "wavelength": 3.565},
                          "jwst.nircam.F410M": {"shorthand": "F410M", "wavelength": 4.083},
                          "jwst.nircam.F444W": {"shorthand": "F444W", "wavelength": 4.402},
                }

Then, we can pass this to an instantiation of the `Catalog` class as the `band_dict` parameter. We will use the small provided set of CIGALE data to demonstrate this:


In [4]:
# Load the astronomical data using the Catalog class
catalog = dataset.Catalog(path="data/cigale_mock_small.csv", 
                          band_dict = NIRCam_band_dict, 
                          summarize = False)

[INFO] Attempting to load file with delimiter: ','
[INFO] Successfully loaded data with 1000 rows.
[INFO] Found 7 valid band columns:
[INFO]   - jwst.nircam.F115W (F115W): 1.154 μm
[INFO]   - jwst.nircam.F150W (F150W): 1.501 μm
[INFO]   - jwst.nircam.F200W (F200W): 1.988 μm
[INFO]   - jwst.nircam.F277W (F277W): 2.776 μm
[INFO]   - jwst.nircam.F356W (F356W): 3.565 μm
[INFO]   - jwst.nircam.F410M (F410M): 4.083 μm
[INFO]   - jwst.nircam.F444W (F444W): 4.402 μm


We can easily see that our catalog object now only uses the 7 NIRCam bands. If we were to then create the default feature dataframe (phots + colors + colors^2), we should only see quantities involving these valid bands. Let's check:

In [5]:
# Create features for modeling
feature_df = catalog.create_feature_dataframe()


[INFO] Created feature dataframe with 49 columns and 1000 rows.
[INFO] Created features are: ['jwst.nircam.F115W', 'jwst.nircam.F150W', 'jwst.nircam.F200W', 'jwst.nircam.F277W', 'jwst.nircam.F356W', 'jwst.nircam.F410M', 'jwst.nircam.F444W', 'F444W/F410M', 'F444W/F356W', 'F444W/F277W', 'F444W/F200W', 'F444W/F150W', 'F444W/F115W', 'F410M/F356W', 'F410M/F277W', 'F410M/F200W', 'F410M/F150W', 'F410M/F115W', 'F356W/F277W', 'F356W/F200W', 'F356W/F150W', 'F356W/F115W', 'F277W/F200W', 'F277W/F150W', 'F277W/F115W', 'F200W/F150W', 'F200W/F115W', 'F150W/F115W', 'F444W/F410M^2', 'F444W/F356W^2', 'F444W/F277W^2', 'F444W/F200W^2', 'F444W/F150W^2', 'F444W/F115W^2', 'F410M/F356W^2', 'F410M/F277W^2', 'F410M/F200W^2', 'F410M/F150W^2', 'F410M/F115W^2', 'F356W/F277W^2', 'F356W/F200W^2', 'F356W/F150W^2', 'F356W/F115W^2', 'F277W/F200W^2', 'F277W/F150W^2', 'F277W/F115W^2', 'F200W/F150W^2', 'F200W/F115W^2', 'F150W/F115W^2']


Good to go! Just remember that the loaded data in the catalog object needs to have the bands you specify!